# Version 2:

## About:
Notebook is used to generate the inputs for the ML model requirements. <br>
The output files for this are:
<ul>
    <li>team_stats.csv - aggregated stats to team level by round</li>
    <li>team_score_results.csv - stacked scored to team level by round</li>
    <li>merged_stat_score_data_clean.csv - combined dataset of aggregated team stats and team scores by round</li>
</ul>

In [1]:
import pandas as pd

In [2]:
from featuresv2 import features, targets, metadata

In [3]:
# read raw data
game_results = pd.read_csv("data/game_results.csv")
player_stats = pd.read_csv("data/player_stats.csv")

# add season field
player_stats['season'] = [i.strftime('%Y') for i in pd.to_datetime(player_stats['utcStartTime']).to_list()]

C:\Users\whitehes\AppData\Local\Temp\ipykernel_13988\3885183847.py:3: DtypeWarning: Columns (49) have mixed types. Specify dtype option on import or set low_memory=False.
  player_stats = pd.read_csv("data/player_stats.csv")


In [4]:
player_stats.head()

,providerId,utcStartTime,status,compSeason.shortName,round.name,round.roundNumber,venue.name,home.team.name,home.team.club.name,away.team.name,...,extendedStats.kickins,extendedStats.kickinsPlayon,player.playerId,player.captain,player.playerJumperNumber,player.givenName,player.surname,teamStatus,team.name,season
0,CD_M20190140101,2019-03-21T08:25:00.000+0000,CONCLUDED,Premiership,Round 1,1,MCG,Carlton,Carlton,Richmond,...,NaN,NaN,CD_I996731,False,30,Charlie,Curnow,home,Carlton,2019
1,CD_M20190140101,2019-03-21T08:25:00.000+0000,CONCLUDED,Premiership,Round 1,1,MCG,Carlton,Carlton,Richmond,...,NaN,NaN,CD_I1000953,False,10,Harry,McKay,home,Carlton,2019
2,CD_M20190140101,2019-03-21T08:25:00.000+0000,CONCLUDED,Premiership,Round 1,1,MCG,Carlton,Carlton,Richmond,...,NaN,NaN,CD_I998215,False,43,Will,Setterfield,home,Carlton,2019
3,CD_M20190140101,2019-03-21T08:25:00.000+0000,CONCLUDED,Premiership,Round 1,1,MCG,Carlton,Carlton,Richmond,...,NaN,NaN,CD_I297255,False,11,Mitch,McGovern,home,Carlton,2019
4,CD_M20190140101,2019-03-21T08:25:00.000+0000,CONCLUDED,Premiership,Round 1,1,MCG,Carlton,Carlton,Richmond,...,NaN,NaN,CD_I230202,False,6,Kade,Simpson,home,Carlton,2019


In [17]:
# define aggregation columns and grouping levels
cols_to_sum = ['goals', 'behinds', 'kicks', 'handballs', 'disposals', 'marks', 'bounces',
       'tackles', 'contestedPossessions', 'uncontestedPossessions',
       'totalPossessions', 'inside50s', 'marksInside50', 'contestedMarks',
       'hitouts', 'onePercenters', 'disposalEfficiency', 'clangers',
       'freesFor', 'freesAgainst', 'rebound50s',
       'goalAssists', 'turnovers', 'intercepts', 'tacklesInside50', 'shotsAtGoal',
       'scoreInvolvements', 'metresGained', 'clearances.centreClearances',
       'clearances.stoppageClearances', 'clearances.totalClearances',
       'extendedStats.effectiveKicks', 'extendedStats.kickToHandballRatio', 'extendedStats.effectiveDisposals',
       'extendedStats.marksOnLead', 'extendedStats.interceptMarks',
       'extendedStats.hitoutsToAdvantage', 'extendedStats.groundBallGets',
       'extendedStats.f50GroundBallGets', 'extendedStats.scoreLaunches',
       'extendedStats.pressureActs', 'extendedStats.defHalfPressureActs',
       'extendedStats.spoils', 'extendedStats.ruckContests',
       'extendedStats.contestDefOneOnOnes', 'extendedStats.contestDefLosses',
       'extendedStats.contestOffOneOnOnes', 'extendedStats.contestOffWins',
       'extendedStats.centreBounceAttendances', 'extendedStats.kickins',
       'extendedStats.kickinsPlayon']
group_by = ['season','round.roundNumber','team.name']
df_cols = cols_to_sum + group_by

In [27]:
# subset to only aggregation + grouping columns
subset_player_stats = player_stats[df_cols]

# apply aggregation and output file
team_stats = subset_player_stats.groupby(group_by).sum()
team_stats.reset_index(inplace=True)
team_stats.to_csv('outputs/team_stats.csv')

In [36]:
score_df = game_results[['round.year', 'round.roundNumber', 'match.homeTeam.name', 'match.awayTeam.name', 'homeTeamScore.matchScore.totalScore','awayTeamScore.matchScore.totalScore']]
score_df['round.year'] = score_df['round.year'].astype(str)
score_df['score_diff'] = score_df['homeTeamScore.matchScore.totalScore'] - score_df['awayTeamScore.matchScore.totalScore']

C:\Users\SeanW\AppData\Local\Temp\ipykernel_19304\1386158496.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  score_df['round.year'] = score_df['round.year'].astype(str)
C:\Users\SeanW\AppData\Local\Temp\ipykernel_19304\1386158496.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  score_df['score_diff'] = score_df['homeTeamScore.matchScore.totalScore'] - score_df['awayTeamScore.matchScore.totalScore']


In [37]:
home_stats = score_df.merge(team_stats, left_on=['round.year', 'round.roundNumber', 'match.homeTeam.name'],
              right_on=['season', 'round.roundNumber', 'team.name'])
homeaway_stats = home_stats.merge(team_stats, left_on=['round.year', 'round.roundNumber', 'match.awayTeam.name'],
              right_on=['season', 'round.roundNumber', 'team.name'], suffixes=('|HOME','|AWAY'))
homeaway_stats.to_csv('outputs/model_training_data_v2.csv')

In [38]:
score_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 7 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   round.year                           999 non-null    object
 1   round.roundNumber                    999 non-null    int64 
 2   match.homeTeam.name                  999 non-null    object
 3   match.awayTeam.name                  999 non-null    object
 4   homeTeamScore.matchScore.totalScore  999 non-null    int64 
 5   awayTeamScore.matchScore.totalScore  999 non-null    int64 
 6   score_diff                           999 non-null    int64 
dtypes: int64(4), object(3)
memory usage: 54.8+ KB


In [39]:
team_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1998 entries, 0 to 1997
Data columns (total 54 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   season                                 1998 non-null   object 
 1   round.roundNumber                      1998 non-null   int64  
 2   team.name                              1998 non-null   object 
 3   goals                                  1998 non-null   int64  
 4   behinds                                1998 non-null   int64  
 5   kicks                                  1998 non-null   int64  
 6   handballs                              1998 non-null   int64  
 7   disposals                              1998 non-null   int64  
 8   marks                                  1998 non-null   int64  
 9   bounces                                1998 non-null   int64  
 10  tackles                                1998 non-null   int64  
 11  cont

In [40]:
homeaway_stats[metadata + features + targets]

,round.year,round.roundNumber,match.homeTeam.name,match.awayTeam.name,kicks|HOME,handballs|HOME,disposals|HOME,marks|HOME,bounces|HOME,tackles|HOME,...,extendedStats.contestDefOneOnOnes|AWAY,extendedStats.contestDefLosses|AWAY,extendedStats.contestOffOneOnOnes|AWAY,extendedStats.contestOffWins|AWAY,extendedStats.centreBounceAttendances|AWAY,extendedStats.kickins|AWAY,extendedStats.kickinsPlayon|AWAY,homeTeamScore.matchScore.totalScore,awayTeamScore.matchScore.totalScore,score_diff
0,2019,1,Carlton,Richmond,237,164,401,99,2,63,...,11,3,24,7,0.0,0.0,0.0,64,97,-33
1,2019,1,Collingwood,Geelong Cats,223,157,380,98,5,49,...,15,3,11,1,0.0,0.0,0.0,65,72,-7
2,2019,1,Melbourne,Port Adelaide,210,123,333,76,7,53,...,9,2,16,7,0.0,0.0,0.0,61,87,-26
3,2019,1,Adelaide Crows,Hawthorn,229,210,439,91,3,58,...,17,1,18,4,0.0,0.0,0.0,55,87,-32
4,2019,1,Western Bulldogs,Sydney Swans,217,157,374,62,6,66,...,25,7,19,7,0.0,0.0,0.0,82,65,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,2023,26,Melbourne,Carlton,228,140,368,97,4,49,...,11,3,11,2,96.0,16.0,13.0,71,73,-2
995,2023,26,Port Adelaide,GWS Giants,173,116,289,63,4,57,...,22,3,13,2,104.0,16.0,16.0,70,93,-23
996,2023,27,Collingwood,GWS Giants,223,133,356,98,4,68,...,19,5,11,3,80.0,10.0,8.0,58,57,1
997,2023,27,Brisbane Lions,Carlton,230,109,339,104,1,53,...,9,1,13,5,96.0,13.0,12.0,79,63,16
